In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import glob
import pandas as pd
import random
import numpy as np
import seaborn as sns
import ast
import json
import os
import re
import itertools
import folium
import hdbscan
import h3

from scipy.spatial.transform import Rotation as R
from scipy.spatial.distance import euclidean
from scipy.spatial.distance import pdist, squareform
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import matplotlib.pylab as pl
import matplotlib.patches as patches
from pydp.algorithms.numerical_mechanisms import LaplaceMechanism
from itertools import combinations
from collections import defaultdict
from collections import Counter
from typing import Dict, Set, List, Optional, Tuple

In [2]:
df_gps = pd.read_csv("C:\\Users\\dmc\\Desktop\\extracted_trips_data.csv")

df_people = pd.read_csv("C:\\Users\\dmc\\Desktop\\individuals_dataset.csv")
df_people = df_people[df_people['GPS_RECORD'] == True]

df_merged = pd.merge(
    df_gps,
    df_people[['ID', 'WEIGHT_INDIV']],
    left_on='person_id',
    right_on='ID',
    how='inner'
).drop(columns='ID')

h3_resolution = 10

df_merged['start_h3'] = df_merged.apply(lambda row: h3.latlng_to_cell(row['start_lat'], row['start_lon'], h3_resolution), axis=1)
df_merged['end_h3'] = df_merged.apply(lambda row: h3.latlng_to_cell(row['end_lat'], row['end_lon'], h3_resolution), axis=1)

media_peso = df_merged['WEIGHT_INDIV'].mean()
print(f"📏 Media del peso individuale: {media_peso:.3f}")

od_matrix_first = df_merged.groupby(['start_h3', 'end_h3']).agg({
    'WEIGHT_INDIV': ['sum', 'count']
}).reset_index()

od_matrix_first.columns = ['start_h3', 'end_h3', 'total_weight', 'count']

📏 Media del peso individuale: 2674.163


In [3]:
# Singolo esagono res=5 : "851fb467fffffff"
# 4 figli di res5 che coprono Parigi centro res=6 : "861fb4667ffffff", "861fb4677ffffff", "861fb466fffffff", "861fb4647ffffff"
# res=7 "871fb4674ffffff", "871fb475bffffff", "871fb4675ffffff", "871fb4666ffffff", "871fb4662ffffff", "871fb4660ffffff"
# "861fb4667ffffff", "861fb4677ffffff", "861fb466fffffff", "861fb4647ffffff", "861fb4297ffffff", "861fb474fffffff", "861fb475fffffff", "861fb462fffffff"
parent_hexes = ["861fb4667ffffff", "861fb4677ffffff", "861fb466fffffff", "861fb4647ffffff", "861fb475fffffff"]

# Genera la lista dei figli a risoluzione 10
target_resolution = 10
start_valid_h3 = set()
end_valid_h3 = set()

for parent in parent_hexes:
    children = h3.cell_to_children(parent, target_resolution)  # <-- NON serve compact
    for child in children:
        start_valid_h3.add(child)
        end_valid_h3.add(child)

mask = (
    (od_matrix_first["start_h3"].isin(start_valid_h3))
    & (od_matrix_first["end_h3"].isin(end_valid_h3))
)

od_matrix_first = od_matrix_first[mask].copy()
print(f"Numero di righe filtrate: {len(od_matrix_first):,}")

Numero di righe filtrate: 23,264


In [4]:
od_matrix = od_matrix_first.copy()
filtered_df = df_merged.merge(
    od_matrix[['start_h3', 'end_h3']],
    on=['start_h3', 'end_h3'],
    how='inner'
)

In [5]:
class H3TreeNode:
    """Nodo dell'albero gerarchico H3"""
    def __init__(self, h3_id: str, resolution: int, total_weight: int = 0, count: int = 0):
        self.h3_id = h3_id
        self.resolution = resolution
        self.total_weight = total_weight
        self.count = count
        self.children: Dict[str, 'H3TreeNode'] = {}
        self.parent: Optional['H3TreeNode'] = None
    
    def add_child(self, child: 'H3TreeNode'):
        """Aggiunge un figlio al nodo"""
        self.children[child.h3_id] = child
        child.parent = self
    
    def add_weight(self, weight: int):
        self.total_weight += weight
        if self.parent:
            self.parent.add_weight(weight)
    
    def add_count(self, count: int):
        self.count += count
        if self.parent:
            self.parent.add_count(count)
    
    def __repr__(self):
        return (f"H3Node(id={self.h3_id}, res={self.resolution}, "
                f"total_weight={self.total_weight}, count={self.count}, children={len(self.children)})")

class H3HierarchicalTree:
    def __init__(self, od_matrix: pd.DataFrame, target_resolution: int = 11, hex_column: str = 'start_h3'):
        self.od_matrix = od_matrix
        self.target_resolution = target_resolution
        self.hex_column = hex_column  # 'start_h3' o 'end_h3'
        self.nodes: Dict[str, H3TreeNode] = {}
        self.root = None
        self.min_resolution = None  # Sarà calcolato dinamicamente
        
    def get_all_hexagons(self) -> Set[str]:
        """Estrae tutti gli esagoni unici dalla colonna specificata"""
        return set(self.od_matrix[self.hex_column].unique())
    
    def get_resolution_coverage(self, hexagons: Set[str], target_res: int) -> Set[str]:
        """
        Ottiene tutti gli esagoni di risoluzione target che coprono l'area
        definita dagli esagoni di input
        """
        coverage_hexagons = set()
        
        for hex_id in hexagons:
            current_res = h3.get_resolution(hex_id)
            
            if current_res == target_res:
                coverage_hexagons.add(hex_id)
            elif current_res < target_res:
                # Dobbiamo espandere a risoluzione più alta
                children = self._get_all_children_at_resolution(hex_id, target_res)
                coverage_hexagons.update(children)
            else:
                # Dobbiamo salire a risoluzione più bassa
                parent = h3.cell_to_parent(hex_id, target_res)
                coverage_hexagons.add(parent)
        
        return coverage_hexagons
    
    def find_optimal_min_resolution(self, hexagons: Set[str]) -> int:
        """
        Trova la risoluzione più alta dove c'è ancora un solo nodo che copre tutti gli esagoni
        """
        print("🔍 Analizzando risoluzione ottimale per radice...")
        
        # Per ogni risoluzione da 0 alla target, conta quanti nodi servono per coprire tutti gli esagoni
        resolution_stats = {}
        
        for resolution in range(0, self.target_resolution + 1):
            ancestors = set()
            for hex_id in hexagons:
                current_res = h3.get_resolution(hex_id)
                if current_res >= resolution:
                    ancestor = h3.cell_to_parent(hex_id, resolution)
                    ancestors.add(ancestor)
                else:
                    # L'esagono è già a risoluzione più bassa, aggiungi direttamente
                    ancestors.add(hex_id)
            
            resolution_stats[resolution] = len(ancestors)
            print(f"  - Risoluzione {resolution}: {len(ancestors)} nodi")
        
        # Trova la risoluzione più alta con count = 1
        optimal_resolution = 0
        for resolution in range(self.target_resolution, -1, -1):
            if resolution_stats[resolution] == 1:
                optimal_resolution = resolution
                break
        
        print(f"✅ Risoluzione ottimale trovata: {optimal_resolution}")
        return optimal_resolution
    
    def get_siblings(self, node_id: str) -> List[str]:
        """Restituisce gli h3_id dei nodi fratelli di node_id (stesso genitore, escluso node_id stesso)"""
        if node_id not in self.nodes:
            return []

        node = self.nodes[node_id]
        parent = node.parent
        if parent is None:
            # Nodo root, non ha fratelli
            return []

        siblings = [child.h3_id for child in parent.children.values() if child.h3_id != node_id]
        return siblings
    
    def get_parent(self, node_id: str) -> Optional[str]:
        """Restituisce l'h3_id del genitore del nodo dato, o None se root o nodo non trovato"""
        if node_id not in self.nodes:
            return None
        node = self.nodes[node_id]
        if node.parent is None:
            return None
        return node.parent.h3_id
    
    def _get_all_children_at_resolution(self, hex_id: str, target_res: int) -> Set[str]:
        """Ricorsivamente ottiene tutti i figli a una risoluzione specifica"""
        current_res = h3.get_resolution(hex_id)
        
        if current_res == target_res:
            return {hex_id}
        elif current_res > target_res:
            # Non dovrebbe succedere in questo contesto
            return set()
        
        children = set()
        direct_children = h3.cell_to_children(hex_id, current_res + 1)
        
        for child in direct_children:
            children.update(self._get_all_children_at_resolution(child, target_res))
        
        return children
    
    def build_hierarchy_path(self, hex_id: str, min_resolution: int) -> List[str]:
        """Costruisce il percorso gerarchico da un esagono fino alla risoluzione minima"""
        path = [hex_id]
        current = hex_id
        current_res = h3.get_resolution(current)
        
        while current_res > min_resolution:
            parent = h3.cell_to_parent(current, current_res - 1)
            path.append(parent)
            current = parent
            current_res -= 1
        
        return path
    
    def create_tree_structure(self):
        """Crea la struttura dell'albero ottimizzato"""
        target_hexagons = self.get_all_hexagons()
        
        # Ottieni copertura completa alla risoluzione target
        coverage_hexagons = self.get_resolution_coverage(target_hexagons, self.target_resolution)
        
        # Trova la risoluzione minima ottimale (quella più alta con count=1)
        self.min_resolution = self.find_optimal_min_resolution(coverage_hexagons)
        
        print(f"🏗️ Costruendo albero da risoluzione {self.min_resolution} a {self.target_resolution}")
        
        # Costruisci tutti i percorsi gerarchici
        all_paths = []
        for hex_id in coverage_hexagons:
            path = self.build_hierarchy_path(hex_id, self.min_resolution)
            all_paths.append(path)
        
        # Crea tutti i nodi
        for path in all_paths:
            for hex_id in path:
                if hex_id not in self.nodes:
                    resolution = h3.get_resolution(hex_id)
                    self.nodes[hex_id] = H3TreeNode(hex_id, resolution)
        
        # Stabilisci relazioni parent-child
        for path in all_paths:
            for i in range(len(path) - 1):
                child_id = path[i]
                parent_id = path[i + 1]
                self.nodes[parent_id].add_child(self.nodes[child_id])
        
        # Identifica la radice
        self.root = self.nodes[self._find_root_hexagon(coverage_hexagons, self.min_resolution)]
        
        return self
    
    def _find_root_hexagon(self, hexagons: Set[str], min_resolution: int) -> str:
        """Trova l'esagono radice che contiene tutti gli altri"""
        sample_hex = next(iter(hexagons))
        return h3.cell_to_parent(sample_hex, min_resolution)
    
    def populate_counts(self):
        # Raggruppa per esagono le due metriche
        agg_df = self.od_matrix.groupby(self.hex_column).agg({'total_weight': 'sum', 'count': 'sum'}).reset_index()
        
        for _, row in agg_df.iterrows():
            hex_id = row[self.hex_column]
            weight = int(row['total_weight'])
            count = int(row['count'])
            
            target_hex = self._map_to_target_resolution(hex_id)
            
            if target_hex in self.nodes:
                self.nodes[target_hex].add_weight(weight)
                self.nodes[target_hex].add_count(count)
            else:
                print(f"⚠️ Avviso: esagono {target_hex} non trovato nell'albero")
        
        return self
    
    def _map_to_target_resolution(self, hex_id: str) -> str:
        """Mappa un esagono alla risoluzione target"""
        current_res = h3.get_resolution(hex_id)
        
        if current_res == self.target_resolution:
            return hex_id
        elif current_res < self.target_resolution:
            # Prendi il primo figlio disponibile (potrebbe essere migliorato)
            children = self._get_all_children_at_resolution(hex_id, self.target_resolution)
            return next(iter(children)) if children else hex_id
        else:
            return h3.cell_to_parent(hex_id, self.target_resolution)
    
    def get_tree_statistics(self) -> Dict:
        """Ottiene statistiche dell'albero"""
        if not self.root:
            return {}
        
        stats = {
            'total_nodes': len(self.nodes),
            'root_resolution': self.root.resolution,
            'min_resolution': self.min_resolution,
            'target_resolution': self.target_resolution,
            'total_weight': self.root.total_weight,
            'nodes_by_resolution': defaultdict(int),
            'resolution_range': f"{self.min_resolution} → {self.target_resolution}"
        }
        
        for node in self.nodes.values():
            stats['nodes_by_resolution'][node.resolution] += 1
        
        return stats
    
    def print_tree(self, max_children_per_level: int = 10):
        if not self.root:
            print("❌ Albero non costruito")
            return
        
        print(f"\n🌳 STRUTTURA ALBERO OTTIMIZZATA")
        print(f"📏 Range risoluzione: {self.min_resolution} → {self.target_resolution}")
        print("=" * 80)
        
        def _print_node(node: H3TreeNode, depth: int = 0, is_last: bool = True, prefix: str = ""):
            connector = "└─ " if is_last else "├─ "
            print(f"{prefix}{connector}{node.h3_id} (res:{node.resolution}, total_weight:{node.total_weight}, count:{node.count}, children:{len(node.children)})")
            
            if is_last:
                child_prefix = prefix + "   "
            else:
                child_prefix = prefix + "│  "
            
            children_list = list(node.children.values())
            
            if len(children_list) <= max_children_per_level:
                for i, child in enumerate(children_list):
                    is_last_child = (i == len(children_list) - 1)
                    _print_node(child, depth + 1, is_last_child, child_prefix)
            else:
                for i in range(max_children_per_level):
                    child = children_list[i]
                    is_last_child = (i == max_children_per_level - 1) and (len(children_list) == max_children_per_level)
                    _print_node(child, depth + 1, is_last_child, child_prefix)
                
                remaining = len(children_list) - max_children_per_level
                print(f"{child_prefix}└─ ... e altri {remaining} figli con stesso pattern")
        
        _print_node(self.root, 0, True, "")

def create_h3_hierarchical_tree(od_matrix_df: pd.DataFrame, target_resolution: int = 10, hex_column: str = 'start_h3'):
    """
    Crea un albero gerarchico H3 ottimizzato dal dataset OD matrix
    
    Args:
        od_matrix_df: DataFrame con colonne 'start_h3', 'end_h3', 'count'
        target_resolution: Risoluzione target per le foglie dell'albero
        hex_column: Colonna da analizzare ('start_h3' o 'end_h3')
    
    Returns:
        H3HierarchicalTree: Albero gerarchico costruito e ottimizzato
    """
    
    tree = H3HierarchicalTree(od_matrix_df, target_resolution, hex_column)
    tree.create_tree_structure()
    tree.populate_counts()
    
    stats = tree.get_tree_statistics()
    print(f"\n📈 STATISTICHE ALBERO OTTIMIZZATO ({hex_column.upper()})")
    print("=" * 50)
    print(f"• Nodi totali: {stats['total_nodes']:,}")
    print(f"• Range risoluzione: {stats['resolution_range']}")
    print(f"• Risoluzione radice: {stats['root_resolution']}")
    print(f"• Risoluzione foglie: {stats['target_resolution']}")
    print(f"• Nodi per risoluzione:")
    for res in sorted(stats['nodes_by_resolution'].keys()):
        print(f"  - Risoluzione {res}: {stats['nodes_by_resolution'][res]:,} nodi")
    
    # Calcola risparmio in nodi
    total_resolutions_possible = target_resolution + 1  # da 0 a target
    resolutions_used = len(stats['nodes_by_resolution'])
    resolutions_saved = total_resolutions_possible - resolutions_used
    
    print(f"\n💡 OTTIMIZZAZIONI:")
    print(f"• Risoluzioni risparmiate: {resolutions_saved}")
    print(f"• Efficienza albero: {resolutions_used}/{total_resolutions_possible} livelli utilizzati")
    
    tree.print_tree()
    
    return tree

In [6]:
tree_start = create_h3_hierarchical_tree(od_matrix, target_resolution=10, hex_column='start_h3')
tree_end = create_h3_hierarchical_tree(od_matrix, target_resolution=10, hex_column='end_h3')

🔍 Analizzando risoluzione ottimale per radice...
  - Risoluzione 0: 1 nodi
  - Risoluzione 1: 1 nodi
  - Risoluzione 2: 1 nodi
  - Risoluzione 3: 1 nodi
  - Risoluzione 4: 1 nodi
  - Risoluzione 5: 2 nodi
  - Risoluzione 6: 5 nodi
  - Risoluzione 7: 35 nodi
  - Risoluzione 8: 244 nodi
  - Risoluzione 9: 1522 nodi
  - Risoluzione 10: 6624 nodi
✅ Risoluzione ottimale trovata: 4
🏗️ Costruendo albero da risoluzione 4 a 10

📈 STATISTICHE ALBERO OTTIMIZZATO (START_H3)
• Nodi totali: 8,433
• Range risoluzione: 4 → 10
• Risoluzione radice: 4
• Risoluzione foglie: 10
• Nodi per risoluzione:
  - Risoluzione 4: 1 nodi
  - Risoluzione 5: 2 nodi
  - Risoluzione 6: 5 nodi
  - Risoluzione 7: 35 nodi
  - Risoluzione 8: 244 nodi
  - Risoluzione 9: 1,522 nodi
  - Risoluzione 10: 6,624 nodi

💡 OTTIMIZZAZIONI:
• Risoluzioni risparmiate: 4
• Efficienza albero: 7/11 livelli utilizzati

🌳 STRUTTURA ALBERO OTTIMIZZATA
📏 Range risoluzione: 4 → 10
└─ 841fb47ffffffff (res:4, total_weight:72646323, count:28203, c

In [7]:
import numpy as np
import pandas as pd
import weakref
from math import *

class H3SbaAggregator:
    def __init__(self, od_matrix, tree_start, tree_end, param):
        # Copia e rinomina colonne
        self.od_matrix = od_matrix.copy()
        self.od_matrix = self.od_matrix.rename(columns={
            'start_h3': 'oi', 
            'end_h3': 'di', 
            'count': 'vol',
            'total_weight': 'weight'
        })
        
        self.tree_start = tree_start
        self.tree_end = tree_end
        self.param = param

        if param.get('target_vol_o') is None:
            raise ValueError('Must set a target_vol_o for tree aggregation')
        self.target_vol_o = param['target_vol_o']
        
        self.anon_thres = param['anon_thres']
        self.suppr_thres_frac = param['suppr_thres_frac']

        # Gestione dei timestep
        if 't' in self.od_matrix.columns:
            vol_ori_df = self.od_matrix.groupby('t')['vol'].sum().reset_index()
        else:
            self.od_matrix['t'] = 0
            vol_ori_df = pd.DataFrame({'t': [0], 'vol': [self.od_matrix['vol'].sum()]})

        self.eval_df = pd.DataFrame(columns=['t', 'vol_ori', 'vol_kept', 'nb_flows'])
        self.od_matrix_agg = pd.DataFrame(columns=['coi', 'cdi', 'vol', 'weight', 't', 'censored'])
        self.od_matrix_agg['censored'] = self.od_matrix_agg['censored'].astype(bool)

        for _, vol_ori_row in vol_ori_df.iterrows():
            timestep = vol_ori_row['t']
            od_matrix_agg_timestep = self.solve_timestep(timestep)

            total_vol = od_matrix_agg_timestep['vol'].sum()
            if total_vol != vol_ori_row['vol']:
                if total_vol >= self.anon_thres:
                    print(f"Warning: missing some volumes! ({od_matrix_agg_timestep['vol'].sum()} vs {vol_ori_row['vol']})")
                else:
                    print('Censored everything as total volume < anon thres')

            self.od_matrix_agg = pd.concat([self.od_matrix_agg, od_matrix_agg_timestep])

            eval_row = {
                't': timestep,
                'vol_ori': vol_ori_row['vol'],
                'vol_kept': (od_matrix_agg_timestep['vol']*(~od_matrix_agg_timestep['censored'])).sum(),
                'nb_flows': len(self.od_matrix_timestep),
            }
            self.eval_df = pd.concat([self.eval_df, pd.DataFrame(eval_row, index=[len(self.eval_df)])])

    def solve_timestep(self, timestep):
        ori_df = self.get_ori_df(timestep)
        total_vol = ori_df.sum()
        
        if total_vol < self.anon_thres:
            return pd.DataFrame({'coi': [self.tree_start.root.h3_id],
                                 'cdi': [self.tree_end.root.h3_id],
                                 'vol': 0,
                                 'weight': 0,
                                 't': timestep,
                                 'censored': total_vol != 0})
        elif total_vol == self.anon_thres:
            return pd.DataFrame({'coi': [self.tree_start.root.h3_id],
                                 'cdi': [self.tree_end.root.h3_id],
                                 'vol': total_vol,
                                 'weight': self.od_matrix_timestep['weight'].sum(),
                                 't': timestep,
                                 'censored': False})
        
        qo = H3VQuadtree(pop_df=ori_df,
                         h3_root=self.tree_start.root,
                         target_vol=self.target_vol_o,
                         ci_col='oi', vol_col='vol')
        o_partition = qo.flat_leaves

        od_matrix_agg_timestep = self.get_dest_agg(o_partition)
        od_matrix_agg_timestep['t'] = timestep
        od_matrix_agg_timestep['censored'] = (od_matrix_agg_timestep['vol'] < self.param['anon_thres']) & (od_matrix_agg_timestep['vol'] != 0)

        return od_matrix_agg_timestep

    def get_dest_agg(self, clusters_o):
        dest_df = self.get_dest_df(clusters_o)

        tree = H3SbaTree(pop_df=dest_df,
                         h3_root=self.tree_end.root,
                         clusters_o=clusters_o,
                         anon_thres=self.anon_thres)

        OD_report = tree.sba_solve(self.suppr_thres_frac * sum([co.vol for co in clusters_o]))
        od_matrix_agg_timestep = pd.DataFrame(OD_report, columns=['coi', 'cdi', 'vol', 'weight'])

        return od_matrix_agg_timestep

    def get_ori_df(self, timestep):
        self.od_matrix_timestep = self.od_matrix[self.od_matrix['t'] == timestep]
        ori_df = self.od_matrix_timestep.groupby('oi')['vol'].sum()
        return ori_df

    def get_dest_df(self, clusters_o):
        nb_leaves_by_cluster_o = [len(cluster_o.flat_leaves_name) for cluster_o in clusters_o]
        oicoidf = pd.DataFrame({
             'oi': np.concatenate([cluster_o.flat_leaves_name for cluster_o in clusters_o]),
             'coi': np.repeat([cluster_o.h3_node.h3_id for cluster_o in clusters_o], nb_leaves_by_cluster_o)
        })

        dest_df = self.od_matrix_timestep.merge(oicoidf, on='oi', how='outer').drop(columns=['oi'])
        dest_df = dest_df.groupby(['coi', 'di']).agg({'vol': 'sum', 'weight': 'sum'}).reset_index()
        dest_df = dest_df.groupby('di').agg(list)

        return dest_df


class H3SbaTree:
    def __init__(self, pop_df, h3_root, clusters_o, anon_thres=0):
        self.pop_df = pop_df
        self.anon_thres = anon_thres
        
        self.clusters_o = clusters_o
        self.cluster_order = {cluster.h3_node.h3_id: i for i, cluster in enumerate(clusters_o)}
        
        self.area = h3_root.resolution
        self.areas_o = np.array([co.h3_node.resolution for co in self.clusters_o])
        
        self.root = H3SbaNode(h3_root, weakref.ref(self))

    def sba_solve(self, S):
        left_delta = 0
        self.root.activate(left_delta)
        left_slope = self.root.best_vol_suppr.sum() - S
        left_score = self.root.best_score.sum() - left_delta*S
        if left_slope < 0:
            return self.get_leaves_arr()

        right_delta = self.area
        self.root.activate(right_delta)
        right_slope = self.root.best_vol_suppr.sum() - S
        right_score = self.root.best_score.sum() - right_delta*S
        
        while right_slope > 0:
            left_delta = right_delta
            left_slope = right_slope
            left_score = right_score
            right_delta += self.area
            self.root.activate(right_delta)
            right_slope = self.root.best_vol_suppr.sum() - S
            right_score = self.root.best_score.sum() - right_delta*S
        
        while True:
            mid_delta = (left_slope*left_delta - right_slope*right_delta + right_score - left_score)/(left_slope - right_slope)
            self.root.activate(mid_delta)
            mid_slope = self.root.best_vol_suppr.sum() - S
            mid_score = self.root.best_score.sum() - mid_delta*S
            
            if round(mid_delta-left_delta, 5) == 0 or round(mid_delta-right_delta, 5) == 0:
                return self.get_leaves_arr()
            else:
                if mid_slope > 0:
                    left_delta = mid_delta
                    left_slope = mid_slope
                    left_score = mid_score
                else:
                    right_delta = mid_delta
                    right_slope = mid_slope
                    right_score = mid_score

    def get_leaves_arr(self):
        leaves_arr = []
        for ori_counter in range(len(self.cluster_order)):
            self.root.get_leaves_arr_rec(leaves_arr, ori_counter)
        return leaves_arr


class H3SbaNode:
    def __init__(self, h3_tree_node, tree_weakref, parent=None):
        self.tree_weakref = tree_weakref
        self.h3_node = h3_tree_node
        self.k = self.tree_weakref().anon_thres
        
        self.children = [H3SbaNode(child, tree_weakref, self) for child in h3_tree_node.children.values()]
        cluster_order = self.tree_weakref().cluster_order
        self.vol_raw = np.zeros(len(cluster_order))
        self.weight_raw = np.zeros(len(cluster_order))
        
        if len(self.children) == 0:
            row = None
            try:
                row = self.tree_weakref().pop_df.loc[self.h3_node.h3_id]
            except KeyError:
                pass
            if row is not None:
                vols = row['vol']
                weights = row['weight']
                cois = row['coi']
                for i in range(len(cois)):
                    self.vol_raw[cluster_order[cois[i]]] = vols[i]
                    self.weight_raw[cluster_order[cois[i]]] = weights[i]
        elif len(self.children) > 0:
            self.vol_raw = np.sum([c.vol_raw for c in self.children], axis=0)
            self.weight_raw = np.sum([c.weight_raw for c in self.children], axis=0)
        
        self.censored = (self.vol_raw > 0) & (self.vol_raw < self.k)
        self.agg_vol_suppr = self.vol_raw * self.censored
        
        self.area = self.h3_node.resolution
        self.agg_error = self.vol_raw * (~self.censored) * (self.tree_weakref().areas_o + self.area)

    def activate(self, delta):
        for c in self.children:
            c.activate(delta)

        self.split_this = np.zeros(len(self.tree_weakref().clusters_o), dtype=bool)
        self.best_vol_suppr = self.agg_vol_suppr.copy()
        self.best_score = self.agg_error + self.agg_vol_suppr * delta

        if len(self.children) > 0:
            split_score = np.sum([c.best_score for c in self.children], axis=0)
            split_mask = (split_score < self.best_score) & (self.vol_raw > self.k)
            self.split_this[split_mask] = True
            self.best_score[split_mask] = split_score[split_mask]
            split_vol_suppr = np.sum([c.best_vol_suppr for c in self.children], axis=0)
            self.best_vol_suppr[split_mask] = split_vol_suppr[split_mask]

    def summary(self, ori_counter):
        cluster_h3_id = self.tree_weakref().clusters_o[ori_counter].h3_node.h3_id
        return [cluster_h3_id, self.h3_node.h3_id, self.vol_raw[ori_counter], self.weight_raw[ori_counter]]

    def get_leaves_arr_rec(self, leaves_arr, ori_counter):
        if not self.split_this[ori_counter]:
            leaves_arr += [self.summary(ori_counter)]
        else:
            for c in self.children:
                c.get_leaves_arr_rec(leaves_arr, ori_counter)


class H3VQuadtree:
    def __init__(self, pop_df, h3_root, target_vol, ci_col, vol_col):
        self.h3_root = h3_root
        self.pop_df = pop_df
        self.target_vol = target_vol
        self.flat_leaves = []
        self.ci_col = ci_col
        self.vol_col = vol_col
        self.grow()

    def grow(self):
        self.root = H3VQuadNode(self.h3_root, self)
        self.root.compute_best_split()
        self.root.keep_subtree(keep_condition=lambda x: x.split_this)
        self.flat_leaves = self.flatten_leaves()

    def flatten_leaves(self):
        flat_leaves = []
        self.root.flatten_leaves_rec(flat_leaves)
        return flat_leaves

    def get_total_error(self):
        return sum([leaf.error for leaf in self.flat_leaves])


class H3VQuadNode:
    def __init__(self, h3_tree_node, tree):
        self.tree = tree
        self.h3_node = h3_tree_node
        self.children = [H3VQuadNode(child, tree) for child in h3_tree_node.children.values()]
        if len(self.children) == 0:
            try:
                self.vol = self.tree.pop_df.loc[self.h3_node.h3_id]
            except KeyError:
                self.vol = 0
        else:
            self.vol = sum([c.vol for c in self.children])
        self.h3_node.vol = self.vol
        if len(self.children) == 0:
            self.flat_leaves_name = [self.h3_node.h3_id]
        else:
            self.flat_leaves_name = []
            for child in self.children:
                self.flat_leaves_name.extend(child.flat_leaves_name)
        self.error = self.compute_error()

    def compute_error(self):
        if self.vol == 0:
            return 0
        return (self.tree.target_vol - self.vol)**2

    def keep_subtree(self, keep_condition):
        if keep_condition(self):
            for c in self.children:
                c.keep_subtree(keep_condition)
        else:
            self.children = []

    def compute_best_split(self):
        self.split_this = False
        self.best_score = self.error
        if self.vol > 0:
            if len(self.children) == 0:
                self.split_this = False
            else:
                for c in self.children:
                    c.compute_best_split()
                split_score = sum([c.best_score for c in self.children])
                if split_score <= self.error:
                    self.split_this = True
                    self.best_score = split_score

    def flatten_leaves_rec(self, flat_leaves):
        if len(self.children) == 0:
            flat_leaves += [self]
        else:
            for c in self.children:
                c.flatten_leaves_rec(flat_leaves)


param = {
    'anon_thres': 10,
    'suppr_thres_frac': 0.1,
    'target_vol_o': 1000,
}

aggregator = H3SbaAggregator(
    od_matrix=od_matrix,
    tree_start=tree_start,
    tree_end=tree_end,
    param=param
)


C:\Users\dmc\AppData\Local\Temp\ipykernel_7208\4068854685.py:50: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.od_matrix_agg = pd.concat([self.od_matrix_agg, od_matrix_agg_timestep])
C:\Users\dmc\AppData\Local\Temp\ipykernel_7208\4068854685.py:58: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.eval_df = pd.concat([self.eval_df, pd.DataFrame(eval_row, index=[len(self.eval_df)])])


In [8]:
od_matrix_agg = aggregator.od_matrix_agg

In [9]:
censored_rows = od_matrix_agg[od_matrix_agg["censored"] == True]
print(censored_rows)

                  coi              cdi  vol        weight  t  censored
2     871fb4640ffffff  871fb4671ffffff  5.0   9371.458158  0      True
3     871fb4640ffffff  871fb4675ffffff  1.0   2558.478648  0      True
4     871fb4640ffffff  871fb4670ffffff  3.0   7383.479119  0      True
6     871fb4640ffffff  871fb4672ffffff  4.0   6794.059745  0      True
8     871fb4640ffffff  871fb4673ffffff  3.0  18021.516601  0      True
...               ...              ...  ...           ... ..       ...
1170  871fb4758ffffff  881fb475cbfffff  4.0  13717.913262  0      True
1171  871fb4758ffffff  881fb475c7fffff  9.0   8533.801954  0      True
1174  871fb4758ffffff  881fb475d5fffff  4.0   7735.509406  0      True
1175  871fb4758ffffff  881fb475ddfffff  1.0   2127.667475  0      True
1178  871fb4758ffffff  881fb475d1fffff  1.0   3303.270685  0      True

[936 rows x 6 columns]


In [10]:
suppressed_count = od_matrix_agg[od_matrix_agg["censored"] == True]["vol"].sum()
print(f"Totale volume soppresso: {suppressed_count}")

Totale volume soppresso: 3087.0


In [11]:
od_matrix_agg = od_matrix_agg[od_matrix_agg["censored"] == False]
od_matrix_agg = od_matrix_agg.rename(columns={
    "coi": "start_h3",
    "cdi": "end_h3",
    "vol": "count"
})

od_matrix_agg = od_matrix_agg.drop(columns=["t", "censored"], errors="ignore")
od_matrix_agg

,start_h3,end_h3,count,weight
0,871fb4644ffffff,841fb47ffffffff,2011.0,5.173206e+06
1,871fb4646ffffff,841fb47ffffffff,1394.0,4.506555e+06
5,871fb4640ffffff,871fb4676ffffff,0.0,0.000000e+00
7,871fb4640ffffff,871fb4674ffffff,0.0,0.000000e+00
10,871fb4640ffffff,881fb4644bfffff,0.0,0.000000e+00
...,...,...,...,...
1177,871fb4758ffffff,881fb475dbfffff,0.0,0.000000e+00
1179,871fb4758ffffff,881fb475d3fffff,0.0,0.000000e+00
1180,871fb475cffffff,841fb47ffffffff,167.0,4.003001e+05
1181,871fb475effffff,841fb47ffffffff,403.0,1.300340e+06


In [12]:
od_matrix_agg[od_matrix_agg['count'] == 0.0]

,start_h3,end_h3,count,weight
5,871fb4640ffffff,871fb4676ffffff,0.0,0.0
7,871fb4640ffffff,871fb4674ffffff,0.0,0.0
10,871fb4640ffffff,881fb4644bfffff,0.0,0.0
11,871fb4640ffffff,881fb4644dfffff,0.0,0.0
16,871fb4640ffffff,8a1fb4644677fff,0.0,0.0
...,...,...,...,...
1169,871fb4758ffffff,881fb475cdfffff,0.0,0.0
1172,871fb4758ffffff,881fb475c1fffff,0.0,0.0
1173,871fb4758ffffff,881fb475d9fffff,0.0,0.0
1177,871fb4758ffffff,881fb475dbfffff,0.0,0.0


In [13]:
od_matrix_agg = od_matrix_agg[od_matrix_agg['count'] > 0.0]

In [14]:
import folium
import h3
import pandas as pd
import numpy as np

class H3FoliumODVisualizer:
    """
    Visualizza i risultati di od_matrix_agg su mappa Folium
    """
    def __init__(self, od_matrix_agg, center_lat=48.8566, center_lon=2.3522):
        """
        Args:
            od_matrix_agg: DataFrame con colonne ['start_h3', 'end_h3', 'count']
                           (eventualmente anche altre colonne come censored/t)
            center_lat, center_lon: centro della mappa (default: Torino)
        """
        self.od_matrix = od_matrix_agg.copy()
        self.center_lat = center_lat
        self.center_lon = center_lon
        
        # Calcola aggregati per origine e destinazione
        self.origin_data = self.od_matrix.groupby("start_h3")["count"].sum().to_dict()
        self.destination_data = self.od_matrix.groupby("end_h3")["count"].sum().to_dict()
        
        # Estrai coppie OD con flussi
        self.od_pairs = list(zip(
            self.od_matrix["start_h3"],
            self.od_matrix["end_h3"],
            self.od_matrix["count"]
        ))
        
        print(f"✅ Estratti {len(self.origin_data)} origini, {len(self.destination_data)} destinazioni, {len(self.od_pairs)} coppie OD")

    def _h3_to_geojson(self, h3_id: str) -> dict:
        """Converte un esagono H3 in GeoJSON"""
        boundary = h3.cell_to_boundary(h3_id)
        coords = [[[lon, lat] for lat, lon in boundary]]
        return {
            "type": "Feature",
            "geometry": {"type": "Polygon", "coordinates": coords},
            "properties": {"h3_id": h3_id, "resolution": h3.get_resolution(h3_id)},
        }

    def create_base_map(self, zoom_start=10) -> folium.Map:
        """Crea la mappa base"""
        m = folium.Map(
            location=[self.center_lat, self.center_lon],
            zoom_start=zoom_start,
            tiles="OpenStreetMap"
        )
        return m
    
    def add_origin_hexagons(self, m: folium.Map, alpha=0.6):
        """Aggiunge esagoni di origine"""
        sorted_origins = sorted(self.origin_data.items(), key=lambda x: x[1], reverse=True)
        if not sorted_origins:
            return
        
        flows = [f for _, f in sorted_origins]
        min_flow, max_flow = min(flows), max(flows)
        group = folium.FeatureGroup(name=f"Origini ({len(sorted_origins)})", show=True)
        
        for rank, (h3_id, flow) in enumerate(sorted_origins, 1):
            try:
                geojson = self._h3_to_geojson(h3_id)
                intensity = (flow - min_flow) / (max_flow - min_flow) if max_flow > min_flow else 1.0
                blue_intensity = int(255 * (0.3 + intensity * 0.7))
                fill_color = f"#{0:02x}{0:02x}{blue_intensity:02x}"
                
                folium.GeoJson(
                    geojson,
                    style_function=lambda x, c=fill_color: {
                        "fillColor": c,
                        "color": "darkblue",
                        "weight": 1,
                        "fillOpacity": alpha,
                        "opacity": 0.8
                    },
                    tooltip=f"Origine: {flow:,} viaggi",
                    popup=folium.Popup(
                        f"<b>Origine H3</b><br>ID: {h3_id}<br>"
                        f"Risoluzione: {h3.get_resolution(h3_id)}<br>"
                        f"Flusso totale: {flow:,}<br>"
                        f"Rank: {rank}",
                        max_width=250
                    )
                ).add_to(group)
            except Exception as e:
                print(f"⚠️ Errore con origine {h3_id}: {e}")
        
        group.add_to(m)

    def add_destination_hexagons(self, m: folium.Map, alpha=0.6):
        """Aggiunge esagoni di destinazione"""
        sorted_dests = sorted(self.destination_data.items(), key=lambda x: x[1], reverse=True)
        if not sorted_dests:
            return
        
        flows = [f for _, f in sorted_dests]
        min_flow, max_flow = min(flows), max(flows)
        group = folium.FeatureGroup(name=f"Destinazioni ({len(sorted_dests)})", show=True)
        
        for rank, (h3_id, flow) in enumerate(sorted_dests, 1):
            try:
                geojson = self._h3_to_geojson(h3_id)
                intensity = (flow - min_flow) / (max_flow - min_flow) if max_flow > min_flow else 1.0
                red_intensity = int(255 * (0.3 + intensity * 0.7))
                fill_color = f"#{red_intensity:02x}{0:02x}{0:02x}"
                
                folium.GeoJson(
                    geojson,
                    style_function=lambda x, c=fill_color: {
                        "fillColor": c,
                        "color": "darkred",
                        "weight": 1,
                        "fillOpacity": alpha,
                        "opacity": 0.8
                    },
                    tooltip=f"Destinazione: {flow:,} viaggi",
                    popup=folium.Popup(
                        f"<b>Destinazione H3</b><br>ID: {h3_id}<br>"
                        f"Risoluzione: {h3.get_resolution(h3_id)}<br>"
                        f"Flusso totale: {flow:,}<br>"
                        f"Rank: {rank}",
                        max_width=250
                    )
                ).add_to(group)
            except Exception as e:
                print(f"⚠️ Errore con destinazione {h3_id}: {e}")
        
        group.add_to(m)

In [15]:
visualizer = H3FoliumODVisualizer(od_matrix_agg)

mappa = visualizer.create_base_map(zoom_start=11)

visualizer.add_origin_hexagons(mappa)
visualizer.add_destination_hexagons(mappa)

folium.LayerControl(collapsed=False).add_to(mappa)

mappa


✅ Estratti 34 origini, 29 destinazioni, 54 coppie OD


In [16]:
import numpy as np
import pandas as pd

def compute_discernability_and_cavg(df: pd.DataFrame, k: int, suppressed_count: int = 0) -> dict:
    """
    Calcola C_DM e C_AVG per un dataset OD generalizzato.
    
    Args:
        df: DataFrame con colonne ['start_h3', 'end_h3', 'count']
        k: soglia k-anonimity
        suppressed_count: numero di coppie OD soppressi (facoltativo)
    
    Returns:
        dict con C_DM, C_AVG, numero totale record e classi equivalenza
    """
    counts = df['count'].values
    total_records = counts.sum() + suppressed_count
    total_equiv_classes = len(counts) + suppressed_count
    total_records_avg = counts.sum()
    total_equiv_classes_avg = len(counts)
    
    # C_DM: somma dei quadrati dei count >= k
    k_anonymous_counts = counts[counts >= k]
    c_dm_gen = np.sum(k_anonymous_counts**2)
    
    # Penalità per record soppressi
    suppression_penalty = suppressed_count * counts.sum() # ogni record soppresso "costa" quanto l'intero dataset
    c_dm = c_dm_gen + suppression_penalty
    
    # C_AVG: (total_records / total_equiv_classes) / k
    c_avg = (total_records_avg / total_equiv_classes_avg) / k if total_equiv_classes_avg > 0 else float('inf')
    
    return {
        'C_DM': c_dm,
        'C_AVG': c_avg,
        'total_records': total_records,
        'total_equivalence_classes': total_equiv_classes,
        'k': k
    }

# Esempio di utilizzo
metrics = compute_discernability_and_cavg(od_matrix_agg, k=10, suppressed_count=suppressed_count)
print("\n📊 Metrics di Discernibilità e CAVG:")
print(f"C_DM: {metrics['C_DM']:,}")
print(f"C_AVG: {metrics['C_AVG']:.4f}")


📊 Metrics di Discernibilità e CAVG:
C_DM: 111,826,844.0
C_AVG: 46.5111


In [17]:
from geopy.distance import geodesic

def calculate_generalization_distance_metric(df: pd.DataFrame, od_matrix_generalized: pd.DataFrame) -> Dict:
   
   print("🔍 Calcolo metrica di distanza post-generalizzazione...")
   
   # 1. Crea mapping da esagoni originali a esagoni generalizzati
   start_original_to_generalized = {}
   end_original_to_generalized = {}
   
   # Ottieni tutti gli esagoni generalizzati unici
   generalized_start_h3 = set(od_matrix_generalized['start_h3'].unique())
   generalized_end_h3 = set(od_matrix_generalized['end_h3'].unique())
   
   # Per ogni esagono originale, trova l'esagono generalizzato corrispondente
   unique_start_h3 = df['start_h3'].unique()
   unique_end_h3 = df['end_h3'].unique()
   
   print(f"📊 Mappatura {len(unique_start_h3)} esagoni origine...")
   for original_h3 in unique_start_h3:
       generalized_h3 = find_generalized_hexagon(original_h3, generalized_start_h3)
       if generalized_h3:
           start_original_to_generalized[original_h3] = generalized_h3
   
   print(f"📊 Mappatura {len(unique_end_h3)} esagoni destinazione...")
   for original_h3 in unique_end_h3:
       generalized_h3 = find_generalized_hexagon(original_h3, generalized_end_h3)
       if generalized_h3:
           end_original_to_generalized[original_h3] = generalized_h3
   
   # 3. Calcola distanze per i punti di partenza
   start_distances = []
   start_coords = []
   
   for idx, row in df.iterrows():
       original_h3 = row['start_h3']
       original_coords = (row['start_lat'], row['start_lon'])
       
       if original_h3 in start_original_to_generalized:
           generalized_h3 = start_original_to_generalized[original_h3]
           generalized_coords = h3.cell_to_latlng(generalized_h3)
           
           distance = geodesic(original_coords, generalized_coords).meters
           
           start_distances.append(distance)
           start_coords.append({
               'original_h3': original_h3,
               'generalized_h3': generalized_h3,
               'original_coords': original_coords,
               'generalized_coords': generalized_coords,
               'distance': distance
           })
   
   # 4. Calcola distanze per i punti di destinazione
   end_distances = []
   end_coords = []
   
   for idx, row in df.iterrows():
       original_h3 = row['end_h3']
       original_coords = (row['end_lat'], row['end_lon'])
       
       if original_h3 in end_original_to_generalized:
           generalized_h3 = end_original_to_generalized[original_h3]
           generalized_coords = h3.cell_to_latlng(generalized_h3)
           
           distance = geodesic(original_coords, generalized_coords).meters
               
           end_distances.append(distance)
           end_coords.append({
               'original_h3': original_h3,
               'generalized_h3': generalized_h3,
               'original_coords': original_coords,
               'generalized_coords': generalized_coords,
               'distance': distance
           })
   
   # 5. Calcola statistiche
   results = {
       'start_distances': {
           'mean': np.mean(start_distances) if start_distances else 0,
           'median': np.median(start_distances) if start_distances else 0,
           'std': np.std(start_distances) if start_distances else 0,
           'min': np.min(start_distances) if start_distances else 0,
           'max': np.max(start_distances) if start_distances else 0,
           'count': len(start_distances)
       },
       'end_distances': {
           'mean': np.mean(end_distances) if end_distances else 0,
           'median': np.median(end_distances) if end_distances else 0,
           'std': np.std(end_distances) if end_distances else 0,
           'min': np.min(end_distances) if end_distances else 0,
           'max': np.max(end_distances) if end_distances else 0,
           'count': len(end_distances)
       },
       'overall': {
           'mean': np.mean(start_distances + end_distances) if (start_distances or end_distances) else 0,
           'median': np.median(start_distances + end_distances) if (start_distances or end_distances) else 0,
           'std': np.std(start_distances + end_distances) if (start_distances or end_distances) else 0,
           'total_points': len(start_distances) + len(end_distances)
       },
       'mappings': {
           'start_original_to_generalized': start_original_to_generalized,
           'end_original_to_generalized': end_original_to_generalized
       },
       'detailed_coords': {
           'start': start_coords,
           'end': end_coords
       }
   }
   
   # 6. Stampa risultati
   print("\n" + "="*60)
   print("📏 METRICHE DI DISTANZA POST-GENERALIZZAZIONE")
   print("="*60)
   
   print(f"\n🎯 PUNTI DI PARTENZA:")
   print(f"   • Distanza media: {results['start_distances']['mean']:.2f} metri")
   print(f"   • Distanza mediana: {results['start_distances']['median']:.2f} metri")
   print(f"   • Deviazione standard: {results['start_distances']['std']:.2f} metri")
   print(f"   • Min-Max: {results['start_distances']['min']:.2f} - {results['start_distances']['max']:.2f} metri")
   print(f"   • Punti analizzati: {results['start_distances']['count']:,}")
   
   print(f"\n🏁 PUNTI DI DESTINAZIONE:")
   print(f"   • Distanza media: {results['end_distances']['mean']:.2f} metri")
   print(f"   • Distanza mediana: {results['end_distances']['median']:.2f} metri")
   print(f"   • Deviazione standard: {results['end_distances']['std']:.2f} metri")
   print(f"   • Min-Max: {results['end_distances']['min']:.2f} - {results['end_distances']['max']:.2f} metri")
   print(f"   • Punti analizzati: {results['end_distances']['count']:,}")
   
   print(f"\n🌍 COMPLESSIVO:")
   print(f"   • Distanza media totale: {results['overall']['mean']:.2f} metri")
   print(f"   • Distanza mediana totale: {results['overall']['median']:.2f} metri")
   print(f"   • Deviazione standard totale: {results['overall']['std']:.2f} metri")
   print(f"   • Punti totali: {results['overall']['total_points']:,}")
   
   return results

def find_generalized_hexagon(original_h3: str, generalized_hexagons: set) -> str:
   """
   Trova l'esagono generalizzato corrispondente a un esagono originale
   """
   # Se l'esagono è già nella lista degli esagoni generalizzati
   if original_h3 in generalized_hexagons:
       return original_h3
   
   # Altrimenti cerca tra tutti gli esagoni generalizzati se l'originale è loro discendente
   for generalized_h3 in generalized_hexagons:
       if is_descendant_of(original_h3, generalized_h3):
           return generalized_h3
   
   return None

def is_descendant_of(child_h3: str, parent_h3: str) -> bool:
   """
   Controlla se child_h3 è discendente di parent_h3
   """
   child_res = h3.get_resolution(child_h3)
   parent_res = h3.get_resolution(parent_h3)
   
   if parent_res >= child_res:
       return False
   
   current = child_h3
   while h3.get_resolution(current) > parent_res:
       current = h3.cell_to_parent(current, h3.get_resolution(current) - 1)
   
   return current == parent_h3

def analyze_generalization_impact(results: Dict) -> None:
   """
   Analizza l'impatto della generalizzazione sulle distanze
   """
   print("\n" + "="*60)
   print("📊 ANALISI IMPATTO GENERALIZZAZIONE")
   print("="*60)
   
   # Calcola percentili
   all_distances = []
   for coord in results['detailed_coords']['start'] + results['detailed_coords']['end']:
       all_distances.append(coord['distance'])
   
   if all_distances:
       percentiles = [25, 50, 75, 90, 95, 99]
       print("\n📏 Distribuzione distanze:")
       for p in percentiles:
           value = np.percentile(all_distances, p)
           print(f"   • {p}° percentile: {value:.2f} metri")
   
   # Analizza per risoluzione
   resolution_analysis = {}
   for coord in results['detailed_coords']['start'] + results['detailed_coords']['end']:
       original_res = h3.get_resolution(coord['original_h3'])
       generalized_res = h3.get_resolution(coord['generalized_h3'])
       
       key = f"{original_res}→{generalized_res}"
       if key not in resolution_analysis:
           resolution_analysis[key] = []
       resolution_analysis[key].append(coord['distance'])
   
   print("\n🔍 Analisi per risoluzione:")
   for resolution_change, distances in resolution_analysis.items():
       mean_dist = np.mean(distances)
       count = len(distances)
       print(f"   • {resolution_change}: {mean_dist:.2f}m (n={count})")

distance_results = calculate_generalization_distance_metric(
   df=filtered_df, 
   od_matrix_generalized=od_matrix_agg
)

analyze_generalization_impact(distance_results)

🔍 Calcolo metrica di distanza post-generalizzazione...
📊 Mappatura 6624 esagoni origine...
📊 Mappatura 6651 esagoni destinazione...


KeyboardInterrupt: 

In [18]:
import pandas as pd
import h3

class GeneralizationMetric:
    """
    Ḡ = (1/V+) × Σ(|o| + |d|) × v_{o→d}
    """
    def __init__(self, k_threshold: int = 10):
        self.k_threshold = k_threshold

    def calculate_generalization_error(self, od_matrix_generalized: pd.DataFrame, od_matrix: pd.DataFrame) -> float:
        # Costruisci dizionari: generalizzato -> numero di celle originali
        origin_counts = self._build_hexagon_counts(
            od_matrix_generalized, od_matrix, column_gen="start_h3", column_orig="start_h3"
        )
        destination_counts = self._build_hexagon_counts(
            od_matrix_generalized, od_matrix, column_gen="end_h3", column_orig="end_h3"
        )

        total_volume_anonymous = 0
        weighted_count_sum = 0

        for _, row in od_matrix_generalized.iterrows():
            flow_value = row["count"]
            if flow_value >= self.k_threshold:
                origin_h3 = row["start_h3"]
                dest_h3   = row["end_h3"]

                origin_count = origin_counts.get(origin_h3, 1)
                dest_count   = destination_counts.get(dest_h3, 1)

                total_volume_anonymous += flow_value
                weighted_count_sum += (origin_count + dest_count) * flow_value

        return weighted_count_sum / total_volume_anonymous if total_volume_anonymous > 0 else 0.0

    def _build_hexagon_counts(
        self, od_matrix_generalized: pd.DataFrame, od_matrix: pd.DataFrame, 
        column_gen: str, column_orig: str
    ) -> dict:
        """
        Conta quanti esagoni originali appartengono a ciascun esagono generalizzato
        """
        generalized_hexagons = od_matrix_generalized[column_gen].unique()
        original_hexagons = od_matrix[column_orig].unique()

        counts = {}
        for gen_hex in generalized_hexagons:
            target_res = h3.get_resolution(gen_hex)

            # Trova tutti i parent degli originali alla risoluzione target
            parent_series = [h3.cell_to_parent(h, target_res) for h in original_hexagons]

            # Conta quante volte compare il parent == gen_hex
            count = sum(1 for p in parent_series if p == gen_hex)
            counts[gen_hex] = max(count, 1)  # fallback a 1

        return counts


# Esempio di utilizzo:
metric = GeneralizationMetric(k_threshold=10)
error = metric.calculate_generalization_error(od_matrix_agg, od_matrix)
print(f"Errore di generalizzazione medio Ḡ: {error:.3f}")

Errore di generalizzazione medio Ḡ: 6807.563


In [30]:
import pandas as pd
import h3
from collections import defaultdict

def fast_reconstruction_loss(original_od_df: pd.DataFrame,
                                       od_matrix_generalized: pd.DataFrame) -> float:
    """
    Calcola la reconstruction loss includendo anche le celle con 0 viaggi.
    Versione ottimizzata, evita itertools.product su tutte le foglie.
    """
    # Dizionario dei flussi originali
    original_flows = {(row['start_h3'], row['end_h3']): row['count']
                      for _, row in original_od_df.iterrows()}

    total_volume = sum(original_flows.values())
    if total_volume == 0:
        return 0.0

    # Dizionario dei flussi generalizzati
    generalized_flows = {(row['start_h3'], row['end_h3']): row['count']
                         for _, row in od_matrix_generalized.iterrows()}

    gen_start_hexes = od_matrix_generalized['start_h3'].unique()
    gen_end_hexes   = od_matrix_generalized['end_h3'].unique()

    # Cache: gen_hex → leaves
    leaf_cache = {}

    def get_leaves(gen_hex, target_res):
        key = (gen_hex, target_res)
        if key in leaf_cache:
            return leaf_cache[key]
        res = h3.get_resolution(gen_hex)
        leaves = {gen_hex} if res == target_res else set(h3.cell_to_children(gen_hex, target_res))
        leaf_cache[key] = leaves
        return leaves

    # Risoluzione target
    target_res_start = h3.get_resolution(original_od_df['start_h3'].iloc[0])
    target_res_end   = h3.get_resolution(original_od_df['end_h3'].iloc[0])

    # Precompute mappe: leaf → parent generalized
    start_leaf_to_parent = {}
    for gen in gen_start_hexes:
        for leaf in get_leaves(gen, target_res_start):
            start_leaf_to_parent[leaf] = gen

    end_leaf_to_parent = {}
    for gen in gen_end_hexes:
        for leaf in get_leaves(gen, target_res_end):
            end_leaf_to_parent[leaf] = gen

    total_abs_error = 0.0

    # Itera solo sulle coppie presenti nell'originale
    for (s, d), true_count in original_flows.items():
        gen_s = start_leaf_to_parent.get(s)
        gen_d = end_leaf_to_parent.get(d)

        if gen_s is None or gen_d is None:
            reconstructed_count = 0.0
        else:
            gen_count = generalized_flows.get((gen_s, gen_d), 0.0)
            start_leaves = get_leaves(gen_s, target_res_start)
            end_leaves   = get_leaves(gen_d, target_res_end)
            reconstructed_count = gen_count / (len(start_leaves) * len(end_leaves))

        total_abs_error += abs(reconstructed_count - true_count)

    # Aggiungi errore per le coppie con count=0 nei generalized flows
    for (gen_s, gen_d), gen_count in generalized_flows.items():
        start_leaves = get_leaves(gen_s, target_res_start)
        end_leaves   = get_leaves(gen_d, target_res_end)
        count_per_leaf = gen_count / (len(start_leaves) * len(end_leaves))

        # Sottrai le coppie già contate
        for s in start_leaves:
            for d in end_leaves:
                if (s, d) not in original_flows:
                    total_abs_error += abs(count_per_leaf - 0.0)

    return total_abs_error / total_volume

loss = fast_reconstruction_loss(
    original_od_df=od_matrix,
    od_matrix_generalized=od_matrix_agg
)
print(f"Reconstruction Loss: {loss:.6f}")

Reconstruction Loss: 1.890015


### Metriche con weight

In [31]:
import numpy as np
import pandas as pd

def compute_discernability_and_cavg(df: pd.DataFrame, k: int, suppressed_count: int = 0) -> dict:
    """
    Calcola C_DM e C_AVG per un dataset OD generalizzato.
    
    Args:
        df: DataFrame con colonne ['start_h3', 'end_h3', 'count']
        k: soglia k-anonimity
        suppressed_count: numero di coppie OD soppressi (facoltativo)
    
    Returns:
        dict con C_DM, C_AVG, numero totale record e classi equivalenza
    """
    counts = df['weight'].values
    total_records = counts.sum() + suppressed_count
    total_equiv_classes = len(counts) + suppressed_count
    total_records_avg = counts.sum()
    total_equiv_classes_avg = len(counts)
    
    # C_DM: somma dei quadrati dei count >= k
    k_anonymous_counts = counts[counts >= k]
    c_dm_gen = np.sum(k_anonymous_counts**2)
    
    # Penalità per record soppressi
    suppression_penalty = suppressed_count * counts.sum() # ogni record soppresso "costa" quanto l'intero dataset
    c_dm = c_dm_gen + suppression_penalty
    
    # C_AVG: (total_records / total_equiv_classes) / k
    c_avg = (total_records_avg / total_equiv_classes_avg) / k if total_equiv_classes_avg > 0 else float('inf')
    
    return {
        'C_DM': c_dm,
        'C_AVG': c_avg,
        'total_records': total_records,
        'total_equivalence_classes': total_equiv_classes,
        'k': k
    }

# Esempio di utilizzo
metrics = compute_discernability_and_cavg(od_matrix_agg, k=10*media_peso, suppressed_count=suppressed_count)
print("\n📊 Metrics di Discernibilità e CAVG:")
print(f"C_DM: {metrics['C_DM']:,}")
print(f"C_AVG: {metrics['C_AVG']:.4f}")


📊 Metrics di Discernibilità e CAVG:
C_DM: 221,856,985,347,188.62
C_AVG: 44.2496


In [32]:
import pandas as pd
import h3

class GeneralizationMetric:
    """
    Ḡ = (1/V+) × Σ(|o| + |d|) × v_{o→d}
    """
    def __init__(self, k_threshold: int = 10):
        self.k_threshold = k_threshold

    def calculate_generalization_error(self, od_matrix_generalized: pd.DataFrame, od_matrix: pd.DataFrame) -> float:
        # Costruisci dizionari: generalizzato -> numero di celle originali
        origin_counts = self._build_hexagon_counts(
            od_matrix_generalized, od_matrix, column_gen="start_h3", column_orig="start_h3"
        )
        destination_counts = self._build_hexagon_counts(
            od_matrix_generalized, od_matrix, column_gen="end_h3", column_orig="end_h3"
        )

        total_volume_anonymous = 0
        weighted_count_sum = 0

        for _, row in od_matrix_generalized.iterrows():
            flow_value = row["weight"]
            if flow_value >= self.k_threshold:
                origin_h3 = row["start_h3"]
                dest_h3   = row["end_h3"]

                origin_count = origin_counts.get(origin_h3, 1)
                dest_count   = destination_counts.get(dest_h3, 1)

                total_volume_anonymous += flow_value
                weighted_count_sum += (origin_count + dest_count) * flow_value

        return weighted_count_sum / total_volume_anonymous if total_volume_anonymous > 0 else 0.0

    def _build_hexagon_counts(
        self, od_matrix_generalized: pd.DataFrame, od_matrix: pd.DataFrame, 
        column_gen: str, column_orig: str
    ) -> dict:
        """
        Conta quanti esagoni originali appartengono a ciascun esagono generalizzato
        """
        generalized_hexagons = od_matrix_generalized[column_gen].unique()
        original_hexagons = od_matrix[column_orig].unique()

        counts = {}
        for gen_hex in generalized_hexagons:
            target_res = h3.get_resolution(gen_hex)

            # Trova tutti i parent degli originali alla risoluzione target
            parent_series = [h3.cell_to_parent(h, target_res) for h in original_hexagons]

            # Conta quante volte compare il parent == gen_hex
            count = sum(1 for p in parent_series if p == gen_hex)
            counts[gen_hex] = max(count, 1)  # fallback a 1

        return counts


# Esempio di utilizzo:
metric = GeneralizationMetric(k_threshold=10*media_peso)
error = metric.calculate_generalization_error(od_matrix_agg, od_matrix)
print(f"Errore di generalizzazione medio Ḡ: {error:.3f}")

Errore di generalizzazione medio Ḡ: 6824.498


In [39]:
import pandas as pd
import h3
from collections import defaultdict

def fast_reconstruction_loss(original_od_df: pd.DataFrame,
                                       od_matrix_generalized: pd.DataFrame) -> float:
    """
    Calcola la reconstruction loss includendo anche le celle con 0 viaggi.
    Versione ottimizzata, evita itertools.product su tutte le foglie.
    """
    # Dizionario dei flussi originali
    original_flows = {(row['start_h3'], row['end_h3']): row['total_weight']
                      for _, row in original_od_df.iterrows()}

    total_volume = sum(original_flows.values())
    if total_volume == 0:
        return 0.0

    # Dizionario dei flussi generalizzati
    generalized_flows = {(row['start_h3'], row['end_h3']): row['weight']
                         for _, row in od_matrix_generalized.iterrows()}

    gen_start_hexes = od_matrix_generalized['start_h3'].unique()
    gen_end_hexes   = od_matrix_generalized['end_h3'].unique()

    # Cache: gen_hex → leaves
    leaf_cache = {}

    def get_leaves(gen_hex, target_res):
        key = (gen_hex, target_res)
        if key in leaf_cache:
            return leaf_cache[key]
        res = h3.get_resolution(gen_hex)
        leaves = {gen_hex} if res == target_res else set(h3.cell_to_children(gen_hex, target_res))
        leaf_cache[key] = leaves
        return leaves

    # Risoluzione target
    target_res_start = h3.get_resolution(original_od_df['start_h3'].iloc[0])
    target_res_end   = h3.get_resolution(original_od_df['end_h3'].iloc[0])

    # Precompute mappe: leaf → parent generalized
    start_leaf_to_parent = {}
    for gen in gen_start_hexes:
        for leaf in get_leaves(gen, target_res_start):
            start_leaf_to_parent[leaf] = gen

    end_leaf_to_parent = {}
    for gen in gen_end_hexes:
        for leaf in get_leaves(gen, target_res_end):
            end_leaf_to_parent[leaf] = gen

    total_abs_error = 0.0

    # Itera solo sulle coppie presenti nell'originale
    for (s, d), true_count in original_flows.items():
        gen_s = start_leaf_to_parent.get(s)
        gen_d = end_leaf_to_parent.get(d)

        if gen_s is None or gen_d is None:
            reconstructed_count = 0.0
        else:
            gen_count = generalized_flows.get((gen_s, gen_d), 0.0)
            start_leaves = get_leaves(gen_s, target_res_start)
            end_leaves   = get_leaves(gen_d, target_res_end)
            reconstructed_count = gen_count / (len(start_leaves) * len(end_leaves))

        total_abs_error += abs(reconstructed_count - true_count)

    # Aggiungi errore per le coppie con count=0 nei generalized flows
    for (gen_s, gen_d), gen_count in generalized_flows.items():
        start_leaves = get_leaves(gen_s, target_res_start)
        end_leaves   = get_leaves(gen_d, target_res_end)
        count_per_leaf = gen_count / (len(start_leaves) * len(end_leaves))

        # Sottrai le coppie già contate
        for s in start_leaves:
            for d in end_leaves:
                if (s, d) not in original_flows:
                    total_abs_error += abs(count_per_leaf - 0.0)

    return total_abs_error / total_volume

loss = fast_reconstruction_loss(
    original_od_df=od_matrix,
    od_matrix_generalized=od_matrix_agg
)
print(f"Reconstruction Loss: {loss:.6f}")

Reconstruction Loss: 1.879069
